In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone 
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os as os

C:\Users\AS\AppData\Roaming\Python\Python311\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY="323fbf31-904a-4d22-b238-fe1a527f5604"
PINECONE_API_ENV="gcp-starter"

In [3]:
def load_pdf(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents


In [4]:
extracted_data=load_pdf("data/")

In [5]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [6]:
text_chunks=text_split(extracted_data)

In [7]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [8]:
embeddings=download_hugging_face_embeddings()

In [9]:
pinecone.init(api_key=PINECONE_API_KEY,environment=PINECONE_API_ENV)
index_name = "medicalchatbot"
l=[t.page_content for t in text_chunks]
# docsearch = Pinecone.from_texts(l, embeddings, index_name=index_name)

In [10]:
docsearch=Pinecone.from_existing_index(index_name, embeddings)
query = "What is Allergy"
docs=docsearch.similarity_search(query, k=3)
print("Result", docs)

Result [Document(page_content='When thisoccurs, an allergy develops against the offending sub-stance (an allergen.)', metadata={}), Document(page_content='Purpose\nAllergy is a reaction of the immune system. Nor-', metadata={}), Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={})]


In [11]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [12]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [13]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",model_type="llama",config={'max_new_tokens':512,'temperature':0.8})

In [14]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [15]:
# while True:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :  Skin infections, whether deep or topical (dermatophytic), can be treated with creams or ointments (topical antifungal drugs). However, if the infection does not clear up after treatment with these products, systemic antifungal drugs may be necessary. It is important to use clean clothes and bedding, avoid sharing personal items, and practice good hygiene to prevent the spread of infection. Additionally, some doctors may recommend an antiseptic soap or gel for washing the infected areas.
